# ModSim Project 3

Maia Materman and SeungU Lyu

In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

## Question

Jupiter is the second heaviest planetary body in the solar system, right after the sun. Even though its mass is relatively small compared to the sun's mass (it is about 0.001% of the sun's mass), the impact it has on other planets in the solar system is huge. Even now, it attracts most of the asteroids in the solar system so that they cannot come toward the Earth. There was an interesting hypothesis that if Jupiter's mass was about 100 times higher than it was when it was created, it might have started nuclear fusion by itself and evolved into a star instead. If this had happened, the solar system could have become a binary star system which is actually common among many other stars in the universe. 

For this question, we thought it might be really interesting to see what would happen to the trajectories of the sun, Earth and Jupiter if for some reason, Jupiter started the nuclear fusion process and became a star with about the same mass as the sun. We thought the most fascinating trajectory might be Earth's and chose to focus on it in our analysis (but more on that later!) 

## Model

In this model we decided to act as though the solar system had only three planetary masses, that of Earth, the sun and Jupiter. This lead to us examining the interactions of six forces which can be seen in our schematic below.

<img src="Schematic.png" width=300 title="schematic">

This model of the system allowed us to extrapolate a few equations which would make up the majority of our system. All of the equations were based around Newton's Law of Universal Gravitation:

\begin{equation}
    F_g = G \frac{m_1 m_2}{r^2}
\end{equation}

This law was the basis for all of our differential equations for the velocity of bodies in our system. Those equations took this form:

\begin{equation}
    \frac{dV}{dt} = \frac{F_{1\;2}}{m_1}+\frac{F_{1\;3}}{m_1}
\end{equation}

The numbers in this equation which are subscripts represent planetary bodies. For example, 1 could represent the Earth; 2, the sun; and 3, Jupiter.

Before moving forward, it is essential to acknowledge some of the assumptions which these equations and other portions of our model introduce. As evidenced in our schematic, we chose to envision the solar system as if it only had three bodies. While doing this helped reduce some of the complexity of our model, it also makes it less accurate. It is also important to note that we had a bit of freedom as we were creating a model of a system which does not (yet?) exist in the physical space of our solar system. With that in mind, we were able to play with some parameters which have little basis in reality. One of those was the rate at which Jupiter grew in mass. We set it to grow linearly in most of our models, although we did also experiment with having it grow to the mass of the sun as soon as the simulation started as well as having it be as massive as the sun in the initial state. These were all modeling decisions which ended up influencing the outcome of our model (although we can't be sure how much they would impact the accuracy). 


In [ ]:
r_e = 147e9
r_j = 778e9
init_jupiter_mass = 1.898e27

init = State(x_e = r_e, y_e = 0 , vx_e = 0 , vy_e = 29784,
             x_s = 0 , y_s = 0 , vx_s = 0 , vy_s = 0 ,
             x_j = r_j, y_j = 0 , vx_j = 0 , vy_j = 13069 , m_j = init_jupiter_mass)

In [ ]:
system = System(init=init,
                G=6.674e-11,
                m_s=1.989e30, 
                m_e=5.972e24,
                t_0=0,
                t_end= 37840000,
                m_j_init = 1.898e27,
                dt = 1000)

In [ ]:
(1.989e+30 - 1.898e27)/system.t_end

In [ ]:
# Here's a function that computes the force of gravity

def universal_gravitation(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    x_e, y_e, vx_e, vy_e, x_s, y_s, vx_s, vy_s, x_j, y_j, vx_j, vy_j, m_j = state
    unpack(system)
    
    
    cur_position_se = Vector(x_s - x_e, y_s - y_e)
    mag_se = cur_position_se.mag
    
    force_se = G * m_s * m_e / mag_se**2
    direction_se = -cur_position_se.hat()
    
    se_force = direction_se * force_se
    
    
    
    cur_position_sj = Vector(x_s - x_j, y_s - y_j)
    mag_sj = cur_position_sj.mag
    
    force_sj = G * m_s * m_j / mag_sj**2
    direction_sj = -cur_position_sj.hat()
    
    sj_force = direction_sj * force_sj
    
    
    
    cur_position_je = Vector(x_j - x_e, y_j - y_e)
    mag_je = cur_position_je.mag
    
    force_je = G * m_j * m_e / mag_je**2
    direction_je = -cur_position_je.hat()
    
    je_force = direction_je * force_je
    
    
    return je_force, sj_force, se_force

In [ ]:
force = universal_gravitation(init,system)

Another thing we played with and iterated on was the way in which the sun was positioned in our system. In some of our first models, the sun is manipulated just by the forces at play in the system. The latter ones, however, reset the position of the sun each time the update function is called.

In [ ]:
def update_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of y and v
    """
    x_e, y_e, vx_e, vy_e, x_s, y_s, vx_s, vy_s, x_j, y_j, vx_j, vy_j, m_j = state
    unpack(system)    

    je_force, sj_force, se_force = universal_gravitation(state, system)
    dx_edt = vx_e
    dy_edt = vy_e
    dvx_edt = (-je_force.x / m_e) + (-se_force.x / m_e)
    dvy_edt = (-je_force.y / m_e) + (-se_force.y / m_e)
    
    dx_jdt = vx_j
    dy_jdt = vy_j
    dvx_jdt = (je_force.x / m_j) + (-sj_force.x / m_j)
    dvy_jdt = (je_force.y / m_j) + (-sj_force.y / m_j)
    
    dx_sdt = vx_s
    dy_sdt = vy_s
    dvx_sdt = (se_force.x / m_s) + (sj_force.x / m_s)
    dvy_sdt = (se_force.y / m_s) + (sj_force.y / m_s)
    
    x_e += dx_edt * dt
    y_e += dy_edt * dt
    vx_e += dvx_edt * dt
    vy_e += dvy_edt * dt
    
    x_s += dx_sdt * dt
    y_s += dy_sdt * dt
    vx_s += dvx_sdt * dt
    vy_s += dvy_sdt * dt
    
    x_j += dx_jdt * dt
    y_j += dy_jdt * dt
    vx_j += dvx_jdt * dt
    vy_j += dvy_jdt * dt
    if m_j < m_s:
        m_j += 5.250882589210215e+21 * dt
    
    return State(x_e = x_e, y_e = y_e, vx_e = vx_e, vy_e = vy_e, x_s = x_s, y_s = y_s, vx_s = vx_s, vy_s = vy_s, x_j = x_j, y_j = y_j, vx_j = vx_j, vy_j = vy_j, m_j = m_j)

This update function (update_func2) resets the sun's position each time which it is called.

In [ ]:
def update_func2(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of y and v
    """
    x_e, y_e, vx_e, vy_e, x_s, y_s, vx_s, vy_s, x_j, y_j, vx_j, vy_j, m_j = state
    unpack(system)    

    je_force, sj_force, se_force = universal_gravitation(state, system)
    dx_edt = vx_e
    dy_edt = vy_e
    dvx_edt = (-je_force.x / m_e) + (-se_force.x / m_e)
    dvy_edt = (-je_force.y / m_e) + (-se_force.y / m_e)
    
    dx_jdt = vx_j
    dy_jdt = vy_j
    dvx_jdt = (je_force.x / m_j) + (-sj_force.x / m_j)
    dvy_jdt = (je_force.y / m_j) + (-sj_force.y / m_j)
    
    dx_sdt = vx_s
    dy_sdt = vy_s
    dvx_sdt = (se_force.x / m_s) + (sj_force.x / m_s)
    dvy_sdt = (se_force.y / m_s) + (sj_force.y / m_s)
    
    x_e += dx_edt * dt
    y_e += dy_edt * dt
    vx_e += dvx_edt * dt
    vy_e += dvy_edt * dt
    
    x_s += dx_sdt * dt
    y_s += dy_sdt * dt
    vx_s += dvx_sdt * dt
    vy_s += dvy_sdt * dt
    
    x_j += dx_jdt * dt
    y_j += dy_jdt * dt
    vx_j += dvx_jdt * dt
    vy_j += dvy_jdt * dt
    if m_j < m_s:
        m_j += 5.250882589210215e+21 * dt
    
    xs = x_s
    ys = y_s
    
    x_e -= xs
    y_e -= ys
    x_s -= xs
    y_s -= ys
    x_j -= xs
    y_j -= ys
    
    return State(x_e = x_e, y_e = y_e, vx_e = vx_e, vy_e = vy_e, x_s = x_s, y_s = y_s, vx_s = vx_s, vy_s = vy_s, x_j = x_j, y_j = y_j, vx_j = vx_j, vy_j = vy_j, m_j = m_j)

In [ ]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[0] = init
    ts = linrange(t_0, t_end, dt)
    
    for t in ts:
        frame.row[t+dt] = update_func(frame.row[t], t, system)
    
    return frame

In [ ]:
def plot_trajectory(results):
    #plot(results.x_e, results.y_e, label='earth')
    plot(results.x_j, results.y_j, label='jupiter')
    plot(results.x_s, results.y_s, label='sun')

    
    decorate(xlabel='x position (m)',
             ylabel='y position (m)')

In [ ]:
%time results1 = run_simulation(system, update_func)

In [ ]:
plot_trajectory(results1)

In [ ]:
system.init.m_j = system.m_s
%time results1_i = run_simulation(system, update_func)
plot_trajectory(results1_i)

We did some experimenting with the orientation of our planets as well. Each system depicts a different orientation of the bodies, one with the sun, Earth and Jupiter in a straight line on a single side and one with Jupiter and the Earth on opposing sides of the sun. Each of those is shown below.

#### This is the orientation of the bodies on the same side of the sun
<img src="InLine.png" width=300 title="PlanetaryBodiesinLine">

#### This is the orientation of the bodies on opposite sides of the sun
<img src="Opposite.png" width=300 title="PlanetaryBodiesonOppositeSides">

The model in the cell directly below this one shows the masses on a single side, while the system defined about five cells below it models the system with the Earth on one side of the sun and Jupiter on the other. 

In [ ]:
init = State(x_e = r_e, y_e = 0 , vx_e = 0 , vy_e = 29784,
             x_s = 0 , y_s = 0 , vx_s = 0 , vy_s = 0 ,
             x_j = r_j, y_j = 0 , vx_j = 0 , vy_j = 13069 , m_j = init_jupiter_mass)

system = System(init=init,
                G=6.674e-11,
                m_s=1.989e30, 
                m_e=5.972e24,
                t_0=0,
                t_end= 378400000,
                m_j_init = 1.898e27,
                dt = 10000)

In [ ]:
%time results2 = run_simulation(system, update_func)

In [ ]:
plot_trajectory(results2)

In [ ]:
%time results3 = run_simulation(system, update_func2)

In [ ]:
plot_trajectory(results3)

In [ ]:
system.init.m_j = system.m_s
%time results3_i = run_simulation(system, update_func2)

In [ ]:
plot_trajectory(results3_i)

In [ ]:
init = State(x_e = r_e, y_e = 0 , vx_e = 0 , vy_e = 29784,
             x_s = 0 , y_s = 0 , vx_s = 0 , vy_s = 0 ,
             x_j = -r_j, y_j = 0 , vx_j = 0 , vy_j = -13069 , m_j = init_jupiter_mass)

system = System(init=init,
                G=6.674e-11,
                m_s=1.989e30, 
                m_e=5.972e24,
                t_0=0,
                t_end= 378400000,
                m_j_init = 1.898e27,
                dt = 10000)

In [ ]:
%time results4 = run_simulation(system, update_func)
plot_trajectory(results4)

In [ ]:
%time results5 = run_simulation(system, update_func2)
plot_trajectory(results5)

In [ ]:
system.init.m_j = system.m_s
%time results5_i = run_simulation(system, update_func2)
plot_trajectory(results5_i)

In [ ]:
init = State(x_e = r_e, y_e = 0 , vx_e = 0 , vy_e = 29784,
             x_s = 0 , y_s = 0 , vx_s = 0 , vy_s = 0 ,
             x_j = 0, y_j = r_j , vx_j = -13069 , vy_j = 0 , m_j = init_jupiter_mass)

system = System(init=init,
                G=6.674e-11,
                m_s=1.989e30, 
                m_e=5.972e24,
                t_0=0,
                t_end= 378400000,
                m_j_init = 1.898e27,
                dt = 10000)

In [ ]:
%time results6 = run_simulation(system, update_func2)
plot_trajectory(results6)

## Results

The answer to our question is depicted in each plot of the bodies. In most of the models, Earth has a trajectory which appears fairly random. In each model, however, Earth is flung out of the solar system. The time which that takes varies by the factors which we manipulated in our simulation. 

## Interpretation

Our model is definitely far from perfect. Although we look at a few parameters and how they relate to the system, we essentially look at them in isolation. It would have been valuable to sweep over a range of values for each thing which we decided to change in our model. It also would have been interesting to see the time that the Earth was flung out of the solar system, since it does that in every simulation. We also could have done more to specify our question and build our model around it. Our question is fairly broad, so making a more exact question which could have been more easily measured probably would have been beneficial.

This notebook contains very visible iterations on our original model. The first model shown is representative of the first simulation we did and every subsequent plot depicts an iteration on that original idea. We first played with the placement of the sun in the system, then how Jupiter increases in size and finally on the orientation of the planets. 